In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import csv
import time

ORG = "google"
BASE_URL = f"https://github.com/orgs/{ORG}/repositories"
OUTPUT_CSV = "google_repos.csv"

options = webdriver.ChromeOptions()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

repos_data = []
page = 1

try:
    while True:
        url = f"{BASE_URL}?page={page}&type=all"
        print(f"[INFO] ページ {page} を取得中: {url}")

        driver.get(url)
        time.sleep(2)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "a.Title-module__anchor--GmXUE.Title-module__inline--oM0P7")
                )
            )
        except TimeoutException:
            print("[INFO] ページ読めないので終了")
            break

        # 毎回 fresh に取得
        repo_names = driver.find_elements(By.CSS_SELECTOR, "a.Title-module__anchor--GmXUE.Title-module__inline--oM0P7")

        if not repo_names:
            print("[INFO] もうリポジトリなし")
            break

        for i in range(len(repo_names)):
            try:
                repo_names = driver.find_elements(By.CSS_SELECTOR, "a.Title-module__anchor--GmXUE.Title-module__inline--oM0P7")
                name_el = repo_names[i]

                repo_name = name_el.text.strip()
                card = name_el.find_element(By.XPATH, "./ancestor::li[1]")

                try:
                    lang = card.find_element(By.CSS_SELECTOR, "span.ReposListItem-module__Text_4--mkG7R").text.strip()
                except:
                    lang = ""

                try:
                    stars = card.find_element(By.CSS_SELECTOR, 'a[aria-label$="stars"]').text.strip()
                except:
                    stars = ""

                print(repo_name, lang, stars)

                repos_data.append({
                    "name": repo_name,
                    "language": lang,
                    "stars": stars
                })

            except StaleElementReferenceException:
                print("[WARN] StaleElement発生 → スキップ")
                continue

        # Nextページ処理
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"], a[aria-label="Next Page"]')
            current = driver.current_url
            next_btn.click()
            time.sleep(2)

            if driver.current_url == current:
                print("[INFO] 最終ページ")
                break

            page += 1

        except NoSuchElementException:
            print("[INFO] Nextボタンなし → 最終ページ")
            break

    # CSV保存
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["name", "language", "stars"])
        writer.writeheader()
        writer.writerows(repos_data)

    print(f"\n[OK] {len(repos_data)}件 保存完了 → {OUTPUT_CSV}")

finally:
    driver.quit()




[INFO] ページ 1 を取得中: https://github.com/orgs/google/repositories?page=1&type=all
yggdrasil-decision-forests C++ 622
site-kit-wp JavaScript 1.3k
oss-fuzz Shell 12k
brotli TypeScript 14k
compile-testing Java 715
device-infra Java 58
filonov Python 10
perfetto C++ 5k
dwh-migration-tools Java 54
error-prone Java 7.1k
nearby C++ 887
turbine Java 82
orbax Python 455
skia C++ 10k
angle C++ 3.8k
capslock Go 1.1k
koladata C++ 27
or-tools C++ 13k
rrg Rust 59
aarch64-paging Rust 38
dawn C++ 778
scaaml Python 175
adk-go Go 4.7k
syzkaller Go 5.9k
cpu_features C++ 2.6k
highway C++ 5.1k
gemma.cpp C++ 6.6k
dranet Go 146
pl011-uart Rust 6
cddlconv TypeScript 11
[INFO] ページ 2 を取得中: https://github.com/orgs/google/repositories?page=2&type=all
jaxite Python 88
nomulus Java 1.8k
xls C++ 1.4k
safe-mmio Rust 17
flatbuffers C++ 25k
sbom-conformance Go 2
ruy C++ 316
earthengine-api JavaScript 3.1k
XNNPACK C 2.2k
fhir-data-pipes Jupyter Notebook 204
truth Java 2.8k
tunix Python 1.9k
garf Python 11
nsjail C++ 3.6k
b

MaxRetryError: HTTPConnectionPool(host='localhost', port=61150): Max retries exceeded with url: /session/d83da260aab3519f1e08952547e85a33/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11867a270>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [3]:
import sqlite3
import csv

path = ''
db_name = 'google_repos.db'
csv_file = 'google_repos.csv'

# --- 文字スター数変換関数 ---
def convert_stars(value):
    value = value.strip().lower()
    if value.endswith("k"):
        return int(float(value[:-1]) * 1000)
    elif value.endswith("m"):
        return int(float(value[:-1]) * 1000000)
    else:
        return int(value)

try:
    # DB接続
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    # --- テーブル作成 ---
    cur.execute("""
        CREATE TABLE IF NOT EXISTS google_repos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            language TEXT,
            stars INTEGER
        );
    """)

    # --- CSV読み込み ---
    with open(csv_file, encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # ヘッダー除外

        repos = [(row[0], row[1], convert_stars(row[2])) for row in reader]

    # --- INSERT ---
    sql = "INSERT INTO google_repos (name, language, stars) VALUES (?, ?, ?);"
    cur.executemany(sql, repos)

    conn.commit()
    print("🎉 CSV → SQLite 保存成功！")

except sqlite3.Error as e:
    print("⚠ SQLite エラー:", e)

finally:
    conn.close()


🎉 CSV → SQLite 保存成功！


In [5]:
path = ''
db_name = 'google_repos.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    #SQL(RDBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    #データを削除するSQL
    #DELETE FROM テーブル名 WHERE id = 1；
    sql = "DELETE FROM google_repos WHERE id BETWEEN 61 AND 90;"
    #sql = "DELETE FROM google_repos WHERE name = '911カレラ';"
    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()
except sqlite3.Error as e:
    print('エラーが発生しました', e)
finally:
    # DBへの接続を閉じる
    conn.close()